### Niwot Ridge, Colorado, USA

**Station name:** nwot

**Location:** [Niwot Ridge LTER](https://nwt.lternet.edu)

**Archive:** [UNAVCO](https://www.unavco.org)

**Ellipsoidal Coordinates:**

- Latitude: 40.05539 

- Longitude: -105.59053

- Height(m): 3522.729 

[UNAVCO station page](https://www.unavco.org/instrumentation/networks/status/nota/overview/NWOT)



<img src="https://www.unavco.org/data/gps-gnss/lib/images/station_images/NWOT.jpg" width=500/>


In [ ]:
import json
import os
import sys

import ipywidgets as widgets
import pandas as pd
import seaborn as sns;

sns.set_theme(style="whitegrid");
import matplotlib.pyplot as plt

# We are including our repository bin to the system path so that we can import the following python modules
bin_path = os.path.abspath(os.path.join('../../../bin'))
if bin_path not in sys.path:
    sys.path.append(bin_path)

import gnssrefl_helpers

#Making sure environment variables are set - this is required to run the gnssrefl code
exists = gnssrefl_helpers.check_environment()
if exists == False:
    gnssrefl_helpers.set_environment(refl_code="../../..", orbits="../../../orbits", exe="../../../bin/exe")

# Set local variable of refl_code location
refl_code_loc = os.environ['REFL_CODE']

# import gnssrefl functions
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.make_json_input import make_json
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.installexe_cl import installexe

#@formatter:off
%matplotlib inline

In [ ]:
# import the crx2rnx file which is dependant on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ['DOCKER']
except KeyError:
    sys = gnssrefl_helpers.get_sys()
    installexe(sys)

Fast Mode - this will download data weekly instead of daily

In [ ]:
weekly = widgets.Checkbox(value=True, description='Fast Mode', disabled=False, indent=False)
display(weekly)

### Data Summary
Station nwot was originally installed/designed to support GPS reflections research. The site was hosted by the Niwot Ridge LTER.

nwot was made to be taller than the typical geodetic antenna so that it would not be buried by snow. It is about 3 meters above the bare soil surface. Because it was installed to support testing GPS reflections, nwot has always tracked L2C.

For this example, we will focus on the data between 2009-2015.

### Imports

### Make a SNR File and run quickLook

We will start by making a SNR file. The main archive for this dataset only provides the high-quality L2C data in the highrate (1-sec) area. We do not need this sample rate for GPS reflectometry, we will use the "special" archive option where the 1-sec data have been decimated to 15 seconds:

In [ ]:
station = 'nwot'
year = 2014
doy = 270

lat = 40.055
long = -105.591
height = 3522.449

In [ ]:
rinex2snr?

In [ ]:
rinex2snr(station, year, doy, archive='special')

Now let's use this **quickLook** command to get a sense of the quality of the 
reflector height (RH) retrievals. 

First L1:

In [ ]:
values, metrics = quicklook(station, year, doy=doy)

These periodograms are a bit busy in the low RH area. But there are 
nice strong peaks in a few of the quadrants. So we can see from this that there may be some azimuth ranges that we will want to remove for teh analysis.

Now try L2:

In [ ]:
# Plotting using pltscreen=True
values, metrics = quicklook(station, year, doy=doy, fr=2)

This plot will have both L2C and non-L2C. There are failed tracks in the gray that are the non-L2C signals.

See L2C only:

In [ ]:
values, metrics = quicklook(station, year, doy=doy, fr=20)

Those failed tracks are now gone and we will use L2C and not all of L2.

### Make multiple years of SNR files 

We are going to look at the data from installation (Fall 2009) through Spring 2015. To speed things
up I will run 2009 and 2015 separately, while the year 2010 through 2014 can be analyzed in 
one line:

In [ ]:
rinex2snr(station, year=2009, doy=240, doy_end=365, archive='special', weekly=weekly.value)

rinex2snr(station, year=2010, doy=1, doy_end=366, archive='special', year_end=2014, weekly=weekly.value)

rinex2snr(station, year=2015, doy=1, doy_end=120, archive='special', weekly=weekly.value)


### Run gnssir for multiple years
Make a json file for your gnssir analysis:


We will use a peak to noise of about 3 and a spectral peak amplitude of 8. We'll also use the southern quadrants (azimuths 90 through 270). We should note that since L5 was not tracked at this site, we will not include it in the json file. Well use a minimum elevation angle of 7 degrees because this particular receiver had a limit on the number of satellites it could track.

In [ ]:
make_json(station, lat, long, height, e1=7, peak2noise=3.2, ampl=8)

In [ ]:
# This is the json file that was created
json_file = f'{refl_code_loc}/input/nwot.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)
file

Now we will set our azimuth range and the frequencies manually.

In [ ]:
# This is the json file that was created
json_file = f'{refl_code_loc}/input/nwot.json'
with open(json_file, "r") as myfile:
    file = json.load(myfile)
    file['azval'] = [90, 180, 180, 270]
    file['freqs'] = [1, 20]
os.remove(json_file)
with open(json_file, 'w') as f:
    json.dump(file, f, indent=4)

with open(json_file, "r") as myfile:
    file = json.load(myfile)

file

Run gnssir for the years 2009-2015:

In [ ]:
year = 2009
doy = 1
doy_end = 365
year_end = 2009
plot = False
gnssir(station, year, doy, doy_end=doy_end, year_end=year_end, plt=plot, screenstats=False)

### Compute daily average RH values
Now we will use the daily_avg utility to compute RH for each day. A median filter of 0.25 meter is used to eliminate large outliers and a minimum number of tracks will be set to 10. This is relatively low because of the small number of L2C transmitting satellites in the early years of the dataset.

In [ ]:
daily_avg(station, medfilter=.25, ReqTracks=10, year1=2009, year2=2010, plt=False, txtfile='nwot-dailyavg.txt')

In [ ]:
filepath = f'{refl_code_loc}/Files/{station}/{station}_allRH.txt'
data = gnssrefl_helpers.read_rh_files(filepath)

df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])
plt.figure(figsize=(8, 8))
g = sns.scatterplot(x='dates', y='rh', data=df, hue='dates', palette='colorblind', legend=False)
g.set_ylim()
g.set_ylabel('Reflector Height (m)');

This first plot shows all of the reflector heights calculated. Next we can show the number of values used in each daily average.

In [ ]:
plt.figure(figsize=(8, 8))
df_group = df.groupby(['dates']).agg(['count'])
g = sns.scatterplot(data=df_group)
g.set_title('Number of values used in the daily average', size=16);

and then we can plot the daily averages:

In [ ]:
filepath = f'{refl_code_loc}/Files/{station}/{station}-dailyavg.txt'
data = gnssrefl_helpers.read_rh_files(filepath)
df = pd.DataFrame(data, index=None, columns=['dates', 'rh'])

plt.figure(figsize=(8, 8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False)
g.set_ylim(3.4, .5)
g.set_ylabel('Reflector Height (m)');

The GPS site at Niwot Ridge was installed because there was a long-standing experiment 
for measuring snow depth. We therefore have a way to assess
accuracy. We download the *in situ* data from 
the [Niwot Ridge facility](https://portal.edirepository.org/nis/mapbrowse?scope=knb-lter-nwt&identifier=34) and will compare to pole 16.

<code>snow depth = RH_baresoil - RH</code>

We will make an estimate of the bare soil reflector height by taking an average of the reflector heights from august to mid september.

In [ ]:
df['doy'] = pd.to_datetime(df['dates']).dt.dayofyear

# get average ground height without snow
# Using august and mid-september to determine "no snow level"
RH_baresoil = df[(df['doy'] >= 213) & (df['doy'] <= 258)]['rh'].mean()

df['rh'] = RH_baresoil - df['rh']

We will then download the data and plot it over the same time period that we analyzed.

In [ ]:
# in situ data for Niwot Ridge LTER
# https://portal.edirepository.org/nis/mapbrowse?scope=knb-lter-nwt&identifier=34
# this is the file we need - it is also stored at Kristine Larsons github.
data_path = 'https://pasta.lternet.edu/package/data/eml/knb-lter-nwt/31/18/6354b6f6c6d4ef8d6334cb3302644164'
data_path = data_path.replace("https://", "http://")
data = pd.read_csv(data_path, delimiter=',', skiprows=1, dtype='str', usecols=(3, 2, 4), index_col=None)

dt1 = pd.read_csv(data_path, skiprows=1, sep=",", quotechar='"',
                  names=["LTER_site", "local_site", "point_ID", "date", "depth_stake", "depth_n", "depth_e", "depth_s",
                         "depth_w", "mean_depth", "num_meas"],
                  parse_dates=['date'],
                  na_values={'LTER_site': ['NaN'], "local_site": ['NaN'], "point_ID": ['NaN'], "date": ['NaN'],
                             "depth_stake": ['NaN'], "depth_n": ['NaN'], "depth_e": ['NaN'], "depth_s": ['NaN'],
                             "depth_w": ['NaN'], "mean_depth": ['NaN'], "num_meas": ['NaN']})

date_range = pd.date_range('2009-09-01', '2015-05-01')

insitu = dt1.loc[(dt1['date'].isin(date_range)) & (dt1['point_ID'] == 16)]
insitu['depth_stake'] = insitu['depth_stake'].astype(float) / 100

per_date = insitu[['date', 'depth_stake']].groupby(by=['date']).mean()

In [ ]:
plt.figure(figsize=(10, 8))
g = sns.scatterplot(x='dates', y='rh', data=df, legend=False, label='GPS_IR')

g2 = sns.scatterplot(x='date', y='depth_stake', data=insitu, s=80, label='Manual - Pole16')

g.set_ylabel('meters')
g.set_xlabel('Years')
g.set_title('Snow Depth, Niwot Ridge LTR Saddle', size=18)
g.set_ylim(-.05, 3)

plt.show()

Citation for data:

Walker, S., J. Morse, and Niwot Ridge LTER. 2020. Snow depth data for saddle snowfence, 1992 - ongoing. ver 11. Environmental Data Initiative. [https://doi.org/10.6073/pasta/abf23758a2e5194aded95bd18c8cbf27] (Accessed 2021-10-19).


This is one of those cases where we would want to note that pole measurements are not representative of the same
footprint as the GPS measurements. As GPS measurements cover an average over a much larger region.

We do not continue with a more quantitative comparison for this demo as there are at least two 
publications in journals and a PhD Dissertation:

[Gutmann, E., K. M. Larson, M. Williams, F.G. Nievinski, and V. Zavorotny, 
Snow measurement by GPS interferometric reflectometry: an evaluation at Niwot Ridge, Colorado, Hydrologic Processes, Vol. 26, 2951-2961, 2012](https://www.kristinelarson.net/wp-content/uploads/2015/10/GutmannEtAl_2012.pdf)

[Nievinski, F.G. and K.M. Larson, Inverse Modeling of GPS Multipath for Snow Depth Estimation, Part II: Application and Validation, IEEE TGRS, Vol. 52(10), 6564-6573, doi:10.1109/TGRS.2013.2297688, 2014](https://www.kristinelarson.net/wp-content/uploads/2015/10/felipe_inv2_revised.pdf)

[Nievinski, F.G., Ph.D. Dissertation, University of Colorado, 2013](https://www.researchgate.net/publication/258848060_Forward_and_Inverse_Modeling_of_GPS_Multipath_for_Snow_Monitoring)